<a href="https://colab.research.google.com/github/richardOlson/nlp__tranformers/blob/main/squad_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# looking at the squad dataset
# getting some of the imports
import requests
import json
import pandas as pd

import torch


In [2]:
files = ['train-v2.0.json', 'dev-v2.0.json']

In [3]:
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/"

In [4]:
# now getting the files 
for file in files:
  res = requests.get(url + file)
  if res.status_code == 200:
    
    with open(file, mode="wb") as f:
     # writing as a chunks 
     for chunk in res.iter_content(chunk_size=50):
       f.write(chunk)



In [5]:
# reading in one of the files to be used with json
with open("dev-v2.0.json", mode="rb")as f:
  dev = json.load(f)

with open("train-v2.0.json", mode="rb")as f:
  train = json.load(f)

In [6]:
train.keys()

dict_keys(['version', 'data'])

In [7]:
train["data"][0]["title"]

'Beyoncé'

In [8]:
squad_list = []

Getting the data into a format that can be used

In [9]:
# now getting the info out of the train
for paragraph_dict in train["data"]:
  for context_with_qa_dict in paragraph_dict["paragraphs"]:
    context = context_with_qa_dict["context"]
    # doing the looping through the question and the answers dictionaries
    for qa_pair_dict in context_with_qa_dict["qas"]:
      if "answers" in qa_pair_dict and len(qa_pair_dict["answers"]) > 0:
          answer = qa_pair_dict['answers'][0]['text']
      elif "plausible_answers" in qa_pair_dict and len(qa_pair_dict["plausible_answers"]) > 0:
        answer = qa_pair_dict["plausible_answers"][0]["text"]
      else:
        answer = None
      # now making the dictionary that will be added to the list 
      squad_list.append({"context": context, "question": qa_pair_dict["question"], "answer": answer})
        
      


In [10]:
len(squad_list)

130319

In [11]:
# creating the dataframe
df = pd.DataFrame(squad_list)

In [12]:
print(df.shape)
df.head()

(130319, 3)


,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s


In [ ]:
df.loc[0, "context"]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [13]:
# now will save the data into a json type object
with open("train.json", mode="w", )as f:
  json.dump(squad_list, f)

In [14]:
dev_list = []

In [15]:
# doing the dev to make it so that it is in the same format as the 
# train.
for paragraph_dict in dev["data"]:
  for each_context_dict in paragraph_dict["paragraphs"]:
    context = each_context_dict["context"]
    for qa_pair_dict in each_context_dict["qas"]:
      if "answers" in qa_pair_dict and len(qa_pair_dict["answers"]) > 0:
        # doing the looping through the answers
        answer_list = qa_pair_dict["answers"]
      elif "plausible_answers" in qa_pair_dict and len(qa_pair_dict["plausible_answers"]) > 0:
        answer_list = qa_pair_dict["plausible_answers"]
      else:
        answer_list = []
      # now doing the making of the list  
      answer = [item["text"] for item in answer_list]
      # removing any duplicates
      answer = list(set(answer))
      
      # adding to the dev list
      dev_list.append({"context": context, "question": qa_pair_dict["question"], "answer": answer})


In [16]:
len(dev_list)

11873

In [ ]:
# building a dataframe of the dev_list
dev_dataframe = pd.DataFrame(dev_list)
dev_dataframe.head()

,context,question,answer
0,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,[France]
1,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"[in the 10th and 11th centuries, 10th and 11th..."
2,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"[Denmark, Iceland and Norway]"
3,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,[Rollo]
4,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,"[the first half of the 10th century, 10th cent..."


In [17]:
# making a json file of the new formatted dev
with open("dev.json", mode="w")as f:
  json.dump(dev_list, f)

In [18]:
# now making the first model 
! pip install transformers -q
import transformers
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import pipeline

     |████████████████████████████████| 2.6 MB 7.8 MB/s 
     |████████████████████████████████| 636 kB 72.9 MB/s 
     |████████████████████████████████| 3.3 MB 33.9 MB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 


In [19]:
# making the model
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")
tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

In [21]:
# going to do it two ways -- one with using the pipelines and the other without the pipeline in 
# transformers

model.

{'input_ids': tensor([[7, 6, 0, 0, 1],
         [1, 2, 3, 0, 0],
         [0, 0, 0, 4, 5]])}

In [25]:
# the span of the questions that we will send into the tokenizer
my_span = squad_list[:5]
my_span

[{'answer': 'in the late 1990s',
  'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
  'question': 'When did Beyonce start becoming popular?'},
 {'answer': 'singing and dancing',
  'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress

In [26]:
# putting the 
questions = [item["question"] for item in my_span]
contexts = [item["context"] for item in my_span]

In [27]:
type(contexts[0])
ques

str

In [22]:
# making the function that will make the "token_type_ids"
def get_token_type_ids(input_id:list):
  t_index = input_id.index(tokenizer.sep_token_id)
  # everything from 0 upto and including the sep token are in the first segment
  front = [0] * t_index
  back = [1] * (len(input_id) - len(front))
  c = front + back
  return c

In [ ]:
questions

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?']

In [56]:
# doing some encoding of the text
input_ids = tokenizer.encode(questions[0], contexts[0])
t = len(input_ids)
print(f"The type of the input_ids is {type(input_ids)}")
print(f"The length of the input_ids is: {t}")
input_ids

The type of the input_ids is <class 'list'>
The length of the input_ids is: 173


[101,
 1332,
 1225,
 24896,
 1320,
 2093,
 1838,
 2479,
 1927,
 136,
 102,
 24041,
 144,
 22080,
 25384,
 118,
 5007,
 113,
 120,
 100,
 120,
 17775,
 118,
 162,
 11414,
 118,
 1474,
 114,
 113,
 1255,
 1347,
 125,
 117,
 2358,
 114,
 1110,
 1126,
 1237,
 2483,
 117,
 5523,
 117,
 1647,
 2451,
 1105,
 3647,
 119,
 3526,
 1105,
 2120,
 1107,
 4666,
 117,
 2245,
 117,
 1131,
 1982,
 1107,
 1672,
 4241,
 1105,
 5923,
 6025,
 1112,
 170,
 2027,
 117,
 1105,
 3152,
 1106,
 8408,
 1107,
 1103,
 1523,
 3281,
 1112,
 1730,
 2483,
 1104,
 155,
 111,
 139,
 1873,
 118,
 1372,
 16784,
 112,
 188,
 6405,
 119,
 2268,
 15841,
 1118,
 1123,
 1401,
 117,
 15112,
 5773,
 25384,
 117,
 1103,
 1372,
 1245,
 1141,
 1104,
 1103,
 1362,
 112,
 188,
 1436,
 118,
 4147,
 1873,
 2114,
 1104,
 1155,
 1159,
 119,
 2397,
 14938,
 1486,
 1103,
 1836,
 1104,
 24041,
 112,
 188,
 1963,
 1312,
 117,
 20924,
 1193,
 1107,
 2185,
 113,
 1581,
 114,
 117,
 1134,
 1628,
 1123,
 1112,
 170,
 3444,
 2360,
 4529,
 117,
 28

In [50]:
tokenizer.cls_token_id

101

In [68]:
contexts[0]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [69]:
# making a function that will be doing the encoding and print out the 
# question and the answers


def answer_question(question:[list, str], context:[list, str]):
  # single context is a flag to wheather their is just on context for all 
  # the questions
  single_context = True
  imput_ids = None

  if isinstance(context, list):
    single_context = False

  if not isinstance(question, list):
    question = [question]
    
  # running the for loop
  for i, q in enumerate(question):
    if single_context:
      input_ids = tokenizer.encode(context, q)
    else:
      input_ids = tokenizer.encode(context[i], q)
    
    # now getting the list of "token_type_ids"
    token_type_ids = get_token_type_ids(input_ids)
    # running them in the model
    output = model(torch.tensor([input_ids]), 
                   token_type_ids=torch.tensor([token_type_ids]), return_dict=True)
    # getting the span
    start = output.start_logits
    end = output.end_logits
    # creating the span
    answer_start = torch.argmax(start)
    answer_end = torch.argmax(end) + 1 # the + 1 is added to make when doing a 
                                      # a span getting the right length

    # getting the answer for the question
    ans = tokenizer.decode(input_ids[answer_start: answer_end])

    # printing out the question and the answer
    print(q)
    print(ans)

  

In [67]:
# using the function above
answer_question(question= questions[0], context=contexts[0])


When did Beyonce start becoming popular?
2003
